In [48]:
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv(r"./Tweets.csv")
data.head(5)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [49]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["sentiment"], test_size=0.2, stratify=data["sentiment"])

len(X_train), len(X_test)

(21984, 5497)

In [50]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens]
    tokens = " ".join(tokens)
    return tokens

X_train = X_train.fillna("").apply(preprocess)
X_test = X_test.fillna("").apply(preprocess)
print(X_train[:5])

vectorizer = TfidfVectorizer(sublinear_tf = True,
                             use_idf = True,
                             ngram_range=(1, 2))

X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

9068     beat aye music hall . babyy , im like cinnamon...
17740    love like war , ,easy begin , hard end .. fiuu...
15407    ` like hear . unabashedly cool . ` make cool b...
24067                                                  car
18598    thing happened today : younger brother ran one...
Name: text, dtype: object


In [51]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

X_train_vectors = torch.tensor(X_train_vectors.toarray(), dtype=torch.float32)
X_test_vectors = torch.tensor(X_test_vectors.toarray(), dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_vectors, y_train)
test_dataset = TensorDataset(X_test_vectors, y_test)

X_train_vectors.shape

torch.Size([21984, 128400])

In [62]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class Network(nn.Module):
    def __init__(self, first_dim):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(first_dim, 1028),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(1028, 512),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(512, 3),
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = Network(X_train_vectors.shape[1]).to(device)

print(model)

Using mps device
Network(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=128400, out_features=1028, bias=True)
    (1): Dropout(p=0.3, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=1028, out_features=512, bias=True)
    (4): Dropout(p=0.3, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=3, bias=True)
  )
)


In [64]:
N = 100
train_dataloader = DataLoader(train_dataset, batch_size=N)
test_dataloader = DataLoader(test_dataset, batch_size=N)

def train(model, dataloader, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 100 == 0:
            loss, current = loss.item(), i * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(model, dataloader, loss_fn):
    size = len(dataloader.dataset)
    num_N = len(dataloader)
    
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss = loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_N
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [65]:
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, test_dataloader, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 1.093451  [    0/21984]
loss: 1.075225  [10000/21984]
loss: 0.877258  [20000/21984]
Test Error: 
 Accuracy: 59.1%, Avg loss: 0.015549 

Epoch 2
-------------------------------
loss: 0.892603  [    0/21984]
loss: 0.579392  [10000/21984]
loss: 0.388784  [20000/21984]
Test Error: 
 Accuracy: 69.0%, Avg loss: 0.014480 

Epoch 3
-------------------------------
loss: 0.521220  [    0/21984]
loss: 0.172630  [10000/21984]
loss: 0.076385  [20000/21984]
Test Error: 
 Accuracy: 68.4%, Avg loss: 0.017837 

Epoch 4
-------------------------------
loss: 0.184793  [    0/21984]
loss: 0.058085  [10000/21984]
loss: 0.017011  [20000/21984]
Test Error: 
 Accuracy: 66.8%, Avg loss: 0.020092 

Epoch 5
-------------------------------
loss: 0.087310  [    0/21984]
loss: 0.029717  [10000/21984]
loss: 0.008467  [20000/21984]
Test Error: 
 Accuracy: 65.9%, Avg loss: 0.021859 

Done!
